In [2]:
class ThresholdBasedRepresentation:
    def _init_(self):
        self.threshold = 0
        self.signal = []

    def generate_EEG_TBR(self, dataArray):
        if len(dataArray.shape) > 1:
            print("Invalid Data Dimension. Expected 1D Array")
            return np.zeros_like(dataArray)
        dataArray_t = np.concatenate(([0], dataArray))
        dataArray_tminus1 = np.concatenate((dataArray, [0]))
        spike_train = dataArray_t - dataArray_tminus1
        spike_train = np.concatenate(([spike_train[1]], spike_train[1:-1]))
        self.threshold = np.mean(spike_train) + 0.5 * np.std(spike_train)
        spike_train = np.where(spike_train > self.threshold, 1, 0)
        return spike_train, self.threshold

    def generate_EEG_SF(self, dataArray, threshold):
        if len(dataArray.shape) > 1:
            print("Invalid Data Dimension. Expected 1D Array")
            return np.zeros_like(dataArray)
        self.threshold = threshold
        baseline = dataArray[0]
        out = np.zeros_like(dataArray)
        bases = np.zeros_like(dataArray)
        for i, s_t in enumerate(dataArray):
            if s_t >= baseline + self.threshold:
                out[i] = 1
                bases[i] = baseline
                baseline += self.threshold
            elif s_t < baseline - self.threshold:
                out[i] = -1
                bases[i] = baseline
                baseline -= self.threshold
        return out, bases

    def generate_EEG_MW(self, dataArray, n_window, threshold):
        if len(dataArray.shape) > 1:
            print("Invalid Data Dimension. Expected 1D Array")
            return np.zeros_like(dataArray)

        L_original = len(dataArray)
        dataArray = np.concatenate(
            (dataArray, np.zeros(
                np.mod(L_original, n_window)
            ))
        )
        L = len(dataArray)
        self.threshold = threshold
        spike_train = np.zeros_like(dataArray)
        
        for window in np.arange(int(np.divide(L, n_window)-1)):
            dataChunk = dataArray[(window)n_window:(window+1)(n_window)]
            baseline = np.mean(dataChunk)
            for i, s_t in enumerate(dataChunk):
                if s_t >= baseline + self.threshold:
                    spike_train[(window)*n_window + i] = 1
                    baseline += self.threshold
                elif s_t < baseline - self.threshold:
                    spike_train[(window)*n_window + i] = -1
                    baseline -= self.threshold

        return spike_train[0:L_original]

    def generate_EEG_BSA(self, signal, fir, threshold):
        self.signal = np.copy(signal)
        if len(self.signal.shape) > 1:
            print("Invalid Data Dimension. Expected 1D Array")
            return np.zeros_like(self.signal)   
        L = len(self.signal)
        F = len(fir)
        spike_train = np.zeros(L)
        self.threshold = threshold
        for t in range(L - F):
            err1 = 0
            err2 = 0
            for k in range(F):
                err1 += abs(self.signal[t + k] - fir[k])
                err2 += abs(self.signal[t + k - 1])
            if err1 <= (err2 * self.threshold):
                spike_train[t] = 1
                for k in range(F):
                    self.signal[t + k + 1] -= fir[k]     
        return spike_train

SyntaxError: invalid non-printable character U+00A0 (2672176572.py, line 84)

In [1]:
class ThresholdBasedRepresentation:
    def _init_(self):
        self.threshold = 0
        self.signal = []

    def generate_EEG_TBR(self, dataArray):
        if len(dataArray.shape) > 1:
            print("Invalid Data Dimension. Expected 1D Array")
            return np.zeros_like(dataArray)
        dataArray_t = np.concatenate((dataArray, [dataArray[-1]]))
        dataArray_tminus1 = np.concatenate(([dataArray[-1]], dataArray))
        spike_train = dataArray_t - dataArray_tminus1
        spike_train = spike_train[0:-1]
        self.threshold = np.mean(spike_train) + 0.5 * np.std(spike_train)
        spike_train = np.where(spike_train > self.threshold, 1, 0)
        return spike_train, self.threshold

    def generate_EEG_SF(self, dataArray, threshold):
        if len(dataArray.shape) > 1:
            print("Invalid Data Dimension. Expected 1D Array")
            return np.zeros_like(dataArray)
        self.threshold = threshold
        baseline = dataArray[0]
        out = np.zeros_like(dataArray)
        bases = np.zeros_like(dataArray)
        for i, s_t in enumerate(dataArray):
            if s_t >= baseline + self.threshold:
                out[i] = 1
                bases[i] = baseline
                baseline += self.threshold
            elif s_t < baseline - self.threshold:
                out[i] = -1
                bases[i] = baseline
                baseline -= self.threshold
        return out, bases

    def generate_EEG_MW(self, dataArray, n_window, threshold):
        if len(dataArray.shape) > 1:
            print("Invalid Data Dimension. Expected 1D Array")
            return np.zeros_like(dataArray)

        L_original = len(dataArray)
        dataArray = np.concatenate(
            (dataArray, np.zeros(
                np.mod(L_original, n_window)
            ))
        )
        L = len(dataArray)
        self.threshold = threshold
        spike_train = np.zeros_like(dataArray)
        
        for window in np.arange(int(np.divide(L, n_window)-1)):
            dataChunk = dataArray[(window)n_window:(window+1)(n_window)]
            baseline = np.mean(dataChunk)
            for i, s_t in enumerate(dataChunk):
                if s_t >= baseline + self.threshold:
                    spike_train[(window)*n_window + i] = 1
                    baseline += self.threshold
                elif s_t < baseline - self.threshold:
                    spike_train[(window)*n_window + i] = -1
                    baseline -= self.threshold

        return spike_train[0:L_original]

    def generate_EEG_BSA(self, signal, fir, threshold):
        self.signal = np.copy(signal)
        if len(self.signal.shape) > 1:
            print("Invalid Data Dimension. Expected 1D Array")
            return np.zeros_like(self.signal)   
        L = len(self.signal)
        F = len(fir)
        spike_train = np.zeros(L)
        self.threshold = threshold
        for t in range(L - F):
            err1 = 0
            err2 = 0
            for k in range(F):
                err1 += abs(self.signal[t + k] - fir[k])
                err2 += abs(self.signal[t + k - 1])
            if err1 <= (err2 * self.threshold):
                spike_train[t] = 1
                for k in range(F):
                    self.signal[t + k + 1] -= fir[k]     
        return spike_train
    def apply_butterworth_filter(self, data, lowcut, highcut, fs, order=5):
        nyquist = 0.5 * fs
        low = lowcut / nyquist
        high = highcut / nyquist
        b, a = signal.butter(order, [low, high], btype='band')
        y = signal.filtfilt(b, a, data)
        return y 
        
    def decode_time_contrast(self, spike_train, threshold, encode_type):
        self.threshold = threshold
        recon = np.zeros_like(spike_train, dtype=np.float16)
        recon[-1] = self.threshold
        print(threshold)

        if encode_type=='tbr':
            recon = self.apply_butterworth_filter(spike_train, 1, 20, 128, 2)
        else:
            for idx, spike in enumerate(np.copy(spike_train)):
                if spike == 1:
                    recon[idx] = recon[idx-1] + threshold
                elif spike == -1:
                    recon[idx] = recon[idx-1] - threshold
                else:
                    recon[idx] = recon[idx-1]

        return recon

    def decode_BSA(self, spike_train, fir):
        recon = np.convolve(spike_train, fir, mode="same")
        return recon

SyntaxError: invalid non-printable character U+00A0 (780910908.py, line 114)